In [ ]:
from __future__ import print_function, division

from keras.datasets import mnist
from keras.layers import Input, Dense, Reshape, Flatten, Dropout, multiply, concatenate
from keras.layers import BatchNormalization, Activation, Embedding, ZeroPadding2D, Lambda
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from keras.optimizers import Adam
from keras.utils import to_categorical
import keras.backend as K
import os
import matplotlib.pyplot as plt
import cv2
import numpy as np

class INFOGAN():
    def __init__(self):
        self.img_rows = 28
        self.img_cols = 28
        self.channels = 1
        self.num_classes = 10
        self.img_shape = (self.img_rows, self.img_cols, self.channels)
        self.latent_dim = 72


        optimizer = Adam(0.0002, 0.5)
        losses = ['binary_crossentropy', self.mutual_info_loss]

        # Build and the discriminator and recognition network
        self.discriminator, self.auxilliary = self.build_disk_and_q_net()

        self.discriminator.compile(loss=['binary_crossentropy'],
            optimizer=optimizer,
            metrics=['accuracy'])

        # Build and compile the recognition network Q
        self.auxilliary.compile(loss=[self.mutual_info_loss],
            optimizer=optimizer,
            metrics=['accuracy'])

        # Build the generator
        self.generator = self.build_generator()

        # The generator takes noise and the target label as input
        # and generates the corresponding digit of that label
        gen_input = Input(shape=(self.latent_dim,))
        img = self.generator(gen_input)

        # For the combined model we will only train the generator
        self.discriminator.trainable = False

        # The discriminator takes generated image as input and determines validity
        valid = self.discriminator(img)
        # The recognition network produces the label
        target_label = self.auxilliary(img)

        # The combined model  (stacked generator and discriminator)
        self.combined = Model(gen_input, [valid, target_label])
        self.combined.compile(loss=losses,
            optimizer=optimizer)


    def build_generator(self):

        model = Sequential()

        model.add(Dense(128 * 7 * 7, activation="relu", input_dim=self.latent_dim))
        model.add(Reshape((7, 7, 128)))
        model.add(BatchNormalization(momentum=0.8))
        model.add(UpSampling2D())
        model.add(Conv2D(128, kernel_size=3, padding="same"))
        model.add(Activation("relu"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(UpSampling2D())
        model.add(Conv2D(64, kernel_size=3, padding="same"))
        model.add(Activation("relu"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Conv2D(self.channels, kernel_size=3, padding='same'))
        model.add(Activation("tanh"))

        gen_input = Input(shape=(self.latent_dim,))
        img = model(gen_input)

        model.summary()

        return Model(gen_input, img)


    def build_disk_and_q_net(self):

        img = Input(shape=self.img_shape)

        # Shared layers between discriminator and recognition network
        model = Sequential()
        model.add(Conv2D(64, kernel_size=3, strides=2, input_shape=self.img_shape, padding="same"))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Conv2D(128, kernel_size=3, strides=2, padding="same"))
        model.add(ZeroPadding2D(padding=((0,1),(0,1))))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Conv2D(256, kernel_size=3, strides=2, padding="same"))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Conv2D(512, kernel_size=3, strides=2, padding="same"))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Flatten())

        img_embedding = model(img)

        # Discriminator
        validity = Dense(1, activation='sigmoid')(img_embedding)

        # Recognition
        q_net = Dense(128, activation='relu')(img_embedding)
        label = Dense(self.num_classes, activation='softmax')(q_net)

        # Return discriminator and recognition network
        return Model(img, validity), Model(img, label)


    def mutual_info_loss(self, c, c_given_x):
        """The mutual information metric we aim to minimize"""
        eps = 1e-8
        conditional_entropy = K.mean(- K.sum(K.log(c_given_x + eps) * c, axis=1))
        entropy = K.mean(- K.sum(K.log(c + eps) * c, axis=1))

        return conditional_entropy + entropy

    def sample_generator_input(self, batch_size):
        # Generator inputs
        sampled_noise = np.random.normal(0, 1, (batch_size, 62))
        sampled_labels = np.random.randint(0, self.num_classes, batch_size).reshape(-1, 1)
        sampled_labels = to_categorical(sampled_labels, num_classes=self.num_classes)

        return sampled_noise, sampled_labels

    def train(self, epochs, batch_size=128, sample_interval=50):

        # Load the dataset
        (X_train, y_train), (_, _) = mnist.load_data()

        # Rescale -1 to 1
        X_train = (X_train.astype(np.float32) - 127.5) / 127.5
        X_train = np.expand_dims(X_train, axis=3)
        y_train = y_train.reshape(-1, 1)

        # Adversarial ground truths
        valid = np.ones((batch_size, 1))
        fake = np.zeros((batch_size, 1))

        for epoch in range(epochs):

            # ---------------------
            #  Train Discriminator
            # ---------------------

            # Select a random half batch of images
            idx = np.random.randint(0, X_train.shape[0], batch_size)
            imgs = X_train[idx]

            # Sample noise and categorical labels
            sampled_noise, sampled_labels = self.sample_generator_input(batch_size)
            gen_input = np.concatenate((sampled_noise, sampled_labels), axis=1)

            # Generate a half batch of new images
            gen_imgs = self.generator.predict(gen_input)

            # Train on real and generated data
            d_loss_real = self.discriminator.train_on_batch(imgs, valid)
            d_loss_fake = self.discriminator.train_on_batch(gen_imgs, fake)

            # Avg. loss
            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

            # ---------------------
            #  Train Generator and Q-network
            # ---------------------

            g_loss = self.combined.train_on_batch(gen_input, [valid, sampled_labels])

            # Plot the progress
            print ("%d [D loss: %.2f, acc.: %.2f%%] [Q loss: %.2f] [G loss: %.2f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss[1], g_loss[2]))

            # If at save interval => save generated image samples
            if epoch % sample_interval == 0:
                self.sample_images(epoch)

    def sample_images(self, epoch):
        r, c = 10, 10

        fig, axs = plt.subplots(r, c)
        for i in range(c):
            sampled_noise, _ = self.sample_generator_input(c)
            label = to_categorical(np.full(fill_value=i, shape=(r,1)), num_classes=self.num_classes)
            gen_input = np.concatenate((sampled_noise, label), axis=1)
            gen_imgs = self.generator.predict(gen_input)
            gen_imgs = 0.5 * gen_imgs + 0.5
            for j in range(r):
                axs[j,i].imshow(gen_imgs[j,:,:,0], cmap='gray')
                axs[j,i].axis('off')
        if not os.path.isdir('images_infogan'):
            os.mkdir('images_infogan')
        fig.savefig("images_infogan/epoch_%d.png" % epoch)
        plt.close()




if __name__ == '__main__':
    infogan = INFOGAN()
    infogan.train(epochs=50000, batch_size=64, sample_interval=200)


    


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_16 (Dense)             (None, 6272)              457856    
_________________________________________________________________
reshape_4 (Reshape)          (None, 7, 7, 128)         0         
_________________________________________________________________
batch_normalization_22 (Batc (None, 7, 7, 128)         512       
_________________________________________________________________
up_sampling2d_7 (UpSampling2 (None, 14, 14, 128)       0         
_________________________________________________________________
conv2d_26 (Conv2D)           (None, 14, 14, 128)       147584    
_________________________________________________________________
activation_10 (Activation)   (None, 14, 14, 128)       0         
_________________________________________________________________
batch_normalization_23 (Batc (None, 14, 14, 128)       512       
__________

101 [D loss: 0.31, acc.: 85.94%] [Q loss: 2.44] [G loss: 0.83]
102 [D loss: 0.32, acc.: 86.72%] [Q loss: 2.35] [G loss: 1.17]
103 [D loss: 0.37, acc.: 80.47%] [Q loss: 2.27] [G loss: 0.78]
104 [D loss: 0.27, acc.: 88.28%] [Q loss: 2.39] [G loss: 0.86]
105 [D loss: 0.39, acc.: 73.44%] [Q loss: 2.53] [G loss: 0.95]
106 [D loss: 0.31, acc.: 85.16%] [Q loss: 2.47] [G loss: 0.76]
107 [D loss: 0.36, acc.: 81.25%] [Q loss: 2.51] [G loss: 0.82]
108 [D loss: 0.34, acc.: 80.47%] [Q loss: 2.66] [G loss: 0.86]
109 [D loss: 0.43, acc.: 78.91%] [Q loss: 2.56] [G loss: 0.80]
110 [D loss: 0.27, acc.: 92.19%] [Q loss: 2.63] [G loss: 0.65]
111 [D loss: 0.44, acc.: 78.91%] [Q loss: 2.44] [G loss: 0.57]
112 [D loss: 0.35, acc.: 82.81%] [Q loss: 2.66] [G loss: 0.68]
113 [D loss: 0.31, acc.: 87.50%] [Q loss: 2.64] [G loss: 0.55]
114 [D loss: 0.41, acc.: 78.12%] [Q loss: 2.57] [G loss: 0.64]
115 [D loss: 0.35, acc.: 83.59%] [Q loss: 2.47] [G loss: 0.44]
116 [D loss: 0.38, acc.: 71.88%] [Q loss: 2.31] [G loss

233 [D loss: 0.61, acc.: 67.19%] [Q loss: 1.56] [G loss: 0.07]
234 [D loss: 0.65, acc.: 62.50%] [Q loss: 1.44] [G loss: 0.21]
235 [D loss: 0.56, acc.: 71.09%] [Q loss: 1.48] [G loss: 0.15]
236 [D loss: 0.62, acc.: 65.62%] [Q loss: 1.48] [G loss: 0.19]
237 [D loss: 0.62, acc.: 64.84%] [Q loss: 1.55] [G loss: 0.23]
238 [D loss: 0.65, acc.: 62.50%] [Q loss: 1.39] [G loss: 0.12]
239 [D loss: 0.59, acc.: 65.62%] [Q loss: 1.37] [G loss: 0.23]
240 [D loss: 0.74, acc.: 61.72%] [Q loss: 1.42] [G loss: 0.10]
241 [D loss: 0.69, acc.: 60.94%] [Q loss: 1.56] [G loss: 0.13]
242 [D loss: 0.69, acc.: 60.16%] [Q loss: 1.45] [G loss: 0.23]
243 [D loss: 0.66, acc.: 60.94%] [Q loss: 1.63] [G loss: 0.11]
244 [D loss: 0.67, acc.: 64.06%] [Q loss: 1.44] [G loss: 0.26]
245 [D loss: 0.66, acc.: 58.59%] [Q loss: 1.30] [G loss: 0.20]
246 [D loss: 0.61, acc.: 69.53%] [Q loss: 1.37] [G loss: 0.10]
247 [D loss: 0.64, acc.: 57.81%] [Q loss: 1.54] [G loss: 0.16]
248 [D loss: 0.66, acc.: 64.06%] [Q loss: 1.53] [G loss

364 [D loss: 0.66, acc.: 62.50%] [Q loss: 1.05] [G loss: 0.06]
365 [D loss: 0.64, acc.: 63.28%] [Q loss: 1.16] [G loss: 0.14]
366 [D loss: 0.67, acc.: 65.62%] [Q loss: 1.25] [G loss: 0.07]
367 [D loss: 0.66, acc.: 60.16%] [Q loss: 1.27] [G loss: 0.09]
368 [D loss: 0.57, acc.: 69.53%] [Q loss: 1.21] [G loss: 0.04]
369 [D loss: 0.73, acc.: 58.59%] [Q loss: 1.11] [G loss: 0.09]
370 [D loss: 0.61, acc.: 64.06%] [Q loss: 1.08] [G loss: 0.08]
371 [D loss: 0.73, acc.: 57.81%] [Q loss: 1.20] [G loss: 0.18]
372 [D loss: 0.61, acc.: 67.97%] [Q loss: 1.22] [G loss: 0.26]
373 [D loss: 0.67, acc.: 57.81%] [Q loss: 1.19] [G loss: 0.06]
374 [D loss: 0.63, acc.: 64.84%] [Q loss: 1.23] [G loss: 0.06]
375 [D loss: 0.63, acc.: 62.50%] [Q loss: 1.10] [G loss: 0.06]
376 [D loss: 0.71, acc.: 60.94%] [Q loss: 1.09] [G loss: 0.11]
377 [D loss: 0.61, acc.: 67.19%] [Q loss: 1.17] [G loss: 0.14]
378 [D loss: 0.65, acc.: 60.94%] [Q loss: 1.20] [G loss: 0.06]
379 [D loss: 0.69, acc.: 60.94%] [Q loss: 1.13] [G loss

497 [D loss: 0.68, acc.: 56.25%] [Q loss: 1.04] [G loss: 0.10]
498 [D loss: 0.65, acc.: 63.28%] [Q loss: 1.19] [G loss: 0.08]
499 [D loss: 0.71, acc.: 60.16%] [Q loss: 0.97] [G loss: 0.08]
500 [D loss: 0.62, acc.: 67.19%] [Q loss: 1.12] [G loss: 0.06]
501 [D loss: 0.66, acc.: 60.94%] [Q loss: 1.07] [G loss: 0.03]
502 [D loss: 0.72, acc.: 57.03%] [Q loss: 1.05] [G loss: 0.13]
503 [D loss: 0.73, acc.: 59.38%] [Q loss: 0.94] [G loss: 0.09]
504 [D loss: 0.74, acc.: 61.72%] [Q loss: 1.16] [G loss: 0.07]
505 [D loss: 0.67, acc.: 62.50%] [Q loss: 1.17] [G loss: 0.07]
506 [D loss: 0.68, acc.: 62.50%] [Q loss: 1.11] [G loss: 0.05]
507 [D loss: 0.66, acc.: 64.06%] [Q loss: 1.06] [G loss: 0.05]
508 [D loss: 0.68, acc.: 64.84%] [Q loss: 0.97] [G loss: 0.04]
509 [D loss: 0.71, acc.: 60.16%] [Q loss: 0.97] [G loss: 0.06]
510 [D loss: 0.73, acc.: 55.47%] [Q loss: 0.86] [G loss: 0.05]
511 [D loss: 0.77, acc.: 53.91%] [Q loss: 0.99] [G loss: 0.02]
512 [D loss: 0.66, acc.: 64.84%] [Q loss: 0.93] [G loss

...

30457 [D loss: 0.72, acc.: 38.28%] [Q loss: 0.73] [G loss: 0.00]
30458 [D loss: 0.71, acc.: 41.41%] [Q loss: 0.74] [G loss: 0.00]
30459 [D loss: 0.71, acc.: 42.19%] [Q loss: 0.72] [G loss: 0.00]
30460 [D loss: 0.73, acc.: 40.62%] [Q loss: 0.72] [G loss: 0.03]
30461 [D loss: 0.73, acc.: 39.06%] [Q loss: 0.74] [G loss: 0.00]
30462 [D loss: 0.73, acc.: 39.06%] [Q loss: 0.73] [G loss: 0.00]
30463 [D loss: 0.71, acc.: 40.62%] [Q loss: 0.73] [G loss: 0.09]
30464 [D loss: 0.72, acc.: 35.16%] [Q loss: 0.74] [G loss: 0.01]
30465 [D loss: 0.72, acc.: 44.53%] [Q loss: 0.73] [G loss: 0.00]
30466 [D loss: 0.72, acc.: 33.59%] [Q loss: 0.74] [G loss: 0.00]
30467 [D loss: 0.71, acc.: 39.06%] [Q loss: 0.73] [G loss: 0.00]
30468 [D loss: 0.72, acc.: 32.81%] [Q loss: 0.72] [G loss: 0.15]
30469 [D loss: 0.71, acc.: 42.97%] [Q loss: 0.75] [G loss: 0.00]
30470 [D loss: 0.71, acc.: 43.75%] [Q loss: 0.73] [G loss: 0.00]
30471 [D loss: 0.71, acc.: 37.50%] [Q loss: 0.73] [G loss: 0.00]
30472 [D loss: 0.72, acc.

30585 [D loss: 0.71, acc.: 49.22%] [Q loss: 0.73] [G loss: 0.00]
30586 [D loss: 0.70, acc.: 44.53%] [Q loss: 0.75] [G loss: 0.00]
30587 [D loss: 0.70, acc.: 46.88%] [Q loss: 0.73] [G loss: 0.00]
30588 [D loss: 0.71, acc.: 43.75%] [Q loss: 0.71] [G loss: 0.00]
30589 [D loss: 0.70, acc.: 51.56%] [Q loss: 0.73] [G loss: 0.00]
30590 [D loss: 0.71, acc.: 47.66%] [Q loss: 0.73] [G loss: 0.00]
30591 [D loss: 0.71, acc.: 42.97%] [Q loss: 0.72] [G loss: 0.00]
30592 [D loss: 0.71, acc.: 46.09%] [Q loss: 0.71] [G loss: 0.08]
30593 [D loss: 0.72, acc.: 46.09%] [Q loss: 0.71] [G loss: 0.00]
30594 [D loss: 0.71, acc.: 40.62%] [Q loss: 0.73] [G loss: 0.02]
30595 [D loss: 0.72, acc.: 36.72%] [Q loss: 0.73] [G loss: 0.00]
30596 [D loss: 0.73, acc.: 37.50%] [Q loss: 0.72] [G loss: 0.02]
30597 [D loss: 0.72, acc.: 35.16%] [Q loss: 0.74] [G loss: 0.00]
30598 [D loss: 0.70, acc.: 43.75%] [Q loss: 0.72] [G loss: 0.00]
30599 [D loss: 0.71, acc.: 46.09%] [Q loss: 0.74] [G loss: 0.01]
30600 [D loss: 0.72, acc.

30713 [D loss: 0.71, acc.: 45.31%] [Q loss: 0.72] [G loss: 0.00]
30714 [D loss: 0.71, acc.: 43.75%] [Q loss: 0.74] [G loss: 0.01]
30715 [D loss: 0.71, acc.: 42.97%] [Q loss: 0.73] [G loss: 0.00]
30716 [D loss: 0.71, acc.: 47.66%] [Q loss: 0.73] [G loss: 0.00]
30717 [D loss: 0.70, acc.: 46.88%] [Q loss: 0.73] [G loss: 0.00]
30718 [D loss: 0.70, acc.: 46.88%] [Q loss: 0.73] [G loss: 0.00]
30719 [D loss: 0.72, acc.: 43.75%] [Q loss: 0.73] [G loss: 0.11]
30720 [D loss: 0.72, acc.: 40.62%] [Q loss: 0.73] [G loss: 0.05]
30721 [D loss: 0.69, acc.: 53.12%] [Q loss: 0.74] [G loss: 0.00]
30722 [D loss: 0.71, acc.: 52.34%] [Q loss: 0.75] [G loss: 0.00]
30723 [D loss: 0.72, acc.: 43.75%] [Q loss: 0.71] [G loss: 0.00]
30724 [D loss: 0.71, acc.: 44.53%] [Q loss: 0.72] [G loss: 0.00]
30725 [D loss: 0.70, acc.: 47.66%] [Q loss: 0.73] [G loss: 0.00]
30726 [D loss: 0.72, acc.: 39.84%] [Q loss: 0.74] [G loss: 0.00]
30727 [D loss: 0.71, acc.: 45.31%] [Q loss: 0.73] [G loss: 0.00]
30728 [D loss: 0.71, acc.

30841 [D loss: 0.71, acc.: 41.41%] [Q loss: 0.75] [G loss: 0.00]
30842 [D loss: 0.71, acc.: 45.31%] [Q loss: 0.75] [G loss: 0.00]
30843 [D loss: 0.71, acc.: 39.84%] [Q loss: 0.75] [G loss: 0.00]
30844 [D loss: 0.72, acc.: 42.19%] [Q loss: 0.74] [G loss: 0.00]
30845 [D loss: 0.70, acc.: 48.44%] [Q loss: 0.73] [G loss: 0.00]
30846 [D loss: 0.70, acc.: 50.00%] [Q loss: 0.73] [G loss: 0.00]
30847 [D loss: 0.69, acc.: 55.47%] [Q loss: 0.75] [G loss: 0.00]
30848 [D loss: 0.71, acc.: 43.75%] [Q loss: 0.74] [G loss: 0.00]
30849 [D loss: 0.70, acc.: 49.22%] [Q loss: 0.72] [G loss: 0.00]
30850 [D loss: 0.71, acc.: 44.53%] [Q loss: 0.73] [G loss: 0.00]
30851 [D loss: 0.71, acc.: 47.66%] [Q loss: 0.70] [G loss: 0.00]
30852 [D loss: 0.71, acc.: 48.44%] [Q loss: 0.73] [G loss: 0.00]
30853 [D loss: 0.70, acc.: 46.09%] [Q loss: 0.76] [G loss: 0.00]
30854 [D loss: 0.71, acc.: 44.53%] [Q loss: 0.73] [G loss: 0.00]
30855 [D loss: 0.71, acc.: 46.09%] [Q loss: 0.72] [G loss: 0.04]
30856 [D loss: 0.70, acc.

30969 [D loss: 0.71, acc.: 46.09%] [Q loss: 0.75] [G loss: 0.00]
30970 [D loss: 0.71, acc.: 42.19%] [Q loss: 0.75] [G loss: 0.00]
30971 [D loss: 0.72, acc.: 43.75%] [Q loss: 0.75] [G loss: 0.00]
30972 [D loss: 0.72, acc.: 42.97%] [Q loss: 0.73] [G loss: 0.01]
30973 [D loss: 0.71, acc.: 45.31%] [Q loss: 0.73] [G loss: 0.00]
30974 [D loss: 0.72, acc.: 42.19%] [Q loss: 0.74] [G loss: 0.00]
30975 [D loss: 0.72, acc.: 37.50%] [Q loss: 0.72] [G loss: 0.02]
30976 [D loss: 0.70, acc.: 46.09%] [Q loss: 0.74] [G loss: 0.00]
30977 [D loss: 0.73, acc.: 34.38%] [Q loss: 0.73] [G loss: 0.00]
30978 [D loss: 0.72, acc.: 33.59%] [Q loss: 0.72] [G loss: 0.00]
30979 [D loss: 0.71, acc.: 46.88%] [Q loss: 0.73] [G loss: 0.00]
30980 [D loss: 0.72, acc.: 39.84%] [Q loss: 0.72] [G loss: 0.00]
30981 [D loss: 0.72, acc.: 39.84%] [Q loss: 0.75] [G loss: 0.00]
30982 [D loss: 0.71, acc.: 40.62%] [Q loss: 0.74] [G loss: 0.01]
30983 [D loss: 0.72, acc.: 41.41%] [Q loss: 0.72] [G loss: 0.00]
30984 [D loss: 0.72, acc.